# 2D Pseudomode Framework - Basic Usage Tutorial

This notebook demonstrates the basic usage of the 2D Pseudomode Framework for simulating non-Markovian quantum dynamics in 2D materials.

## Installation

First, ensure you have the Python bindings installed:

```bash
pip install pseudomode-framework
```

Or build from source:

```bash
mkdir build && cd build
cmake .. -DBUILD_PYTHON_BINDINGS=ON
make -j$(nproc)
make install
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pseudomode_py as pm

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Define System Parameters

We'll simulate a qubit coupled to phonons in graphene.

In [ ]:
# System parameters
system_params = pm.System2DParams()
system_params.omega0_eV = 1.4  # System frequency (eV)
system_params.temperature_K = 300.0  # Room temperature

# Simulation configuration
config = pm.SimulationConfig()
config.max_pseudomodes = 5
config.adaptive_n_max = 4
config.time_step_ps = 0.01
config.total_time_ps = 50.0
config.use_gpu = True  # Enable GPU acceleration if available

print(f"System frequency: {system_params.omega0_eV} eV")
print(f"Temperature: {system_params.temperature_K} K")
print(f"Max pseudomodes: {config.max_pseudomodes}")

## 2. Build Spectral Density

Define the spectral density function J(ω) for graphene phonons.

In [ ]:
# Frequency grid
omega_grid = np.linspace(0, 5.0, 1000)  # 0 to 5 eV

# Build graphene spectral density
J_omega = pm.SpectralDensity2D.build_material_spectrum(
    omega_grid, "graphene"
)

# Plot spectral density
plt.figure(figsize=(10, 6))
plt.plot(omega_grid, J_omega, linewidth=2)
plt.xlabel('Energy (eV)', fontsize=14)
plt.ylabel('Spectral Density J(ω) (eV)', fontsize=14)
plt.title('Graphene Phonon Spectral Density', fontsize=16)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Peak spectral density: {np.max(J_omega):.4f} eV")
print(f"Peak location: {omega_grid[np.argmax(J_omega)]:.4f} eV")

## 3. Run Pseudomode Simulation

Execute the complete workflow: fitting → dynamics → coherence times.

In [ ]:
# Create framework instance
framework = pm.PseudomodeFramework2D(config)

# Time grid for evolution
time_grid = np.linspace(0, config.total_time_ps, 500)

# Run simulation
print("Running simulation...")
result = framework.simulate_material(
    "graphene",
    system_params,
    omega_grid,
    time_grid
)

print(f"\nSimulation completed in {result.computation_time_seconds:.2f} seconds")
print(f"Status: {result.status}")
print(f"\nFitted pseudomodes: {len(result.fitted_modes)}")

# Display fitted modes
for i, mode in enumerate(result.fitted_modes):
    print(f"  Mode {i+1}: ω = {mode.omega_eV:.4f} eV, γ = {mode.gamma_eV:.4f} eV, g = {mode.g_eV:.4f} eV")

## 4. Extract Coherence Times

Calculate T₁, T₂*, and T₂ from the dynamics.

In [ ]:
coherence = result.coherence_times

print("Coherence Times:")
print(f"  T₁ (relaxation):        {coherence.T1_ps:.2f} ps")
print(f"  T₂* (dephasing):        {coherence.T2_star_ps:.2f} ps")
print(f"  T₂ (echo):              {coherence.T2_echo_ps:.2f} ps")

# Visualize coherence times
fig, ax = plt.subplots(figsize=(8, 6))
times = [coherence.T1_ps, coherence.T2_star_ps, coherence.T2_echo_ps]
labels = ['T₁', 'T₂*', 'T₂']
colors = ['#e74c3c', '#3498db', '#2ecc71']

ax.bar(labels, times, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
ax.set_ylabel('Time (ps)', fontsize=14)
ax.set_title('Coherence Times in Graphene', fontsize=16)
ax.grid(True, axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Analyze Time Evolution

Plot population and coherence dynamics.

In [ ]:
# Extract populations and coherences from time evolution
populations = []
coherences = []

for state in result.time_evolution:
    # Get reduced density matrix (system only)
    rho_sys = state.partial_trace_system()
    
    # Extract populations (diagonal elements)
    # This is pseudo-code - actual implementation depends on API
    pop_0 = abs(rho_sys[0, 0])
    populations.append(pop_0)
    
    # Extract coherence (off-diagonal element)
    coh = abs(rho_sys[0, 1])
    coherences.append(coh)

# Plot dynamics
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Population dynamics
ax1.plot(time_grid, populations, linewidth=2, label='P₀(t)')
ax1.plot(time_grid, 1 - np.array(populations), linewidth=2, label='P₁(t)')
ax1.set_xlabel('Time (ps)', fontsize=12)
ax1.set_ylabel('Population', fontsize=12)
ax1.set_title('Population Dynamics', fontsize=14)
ax1.legend(fontsize=12)
ax1.grid(True, alpha=0.3)

# Coherence dynamics
ax2.plot(time_grid, coherences, linewidth=2, color='#e74c3c')
ax2.set_xlabel('Time (ps)', fontsize=12)
ax2.set_ylabel('|ρ₀₁(t)|', fontsize=12)
ax2.set_title('Coherence Decay', fontsize=14)
ax2.grid(True, alpha=0.3)

# Fit exponential decay
from scipy.optimize import curve_fit

def exp_decay(t, A, T):
    return A * np.exp(-t / T)

popt, _ = curve_fit(exp_decay, time_grid, coherences, p0=[1.0, 10.0])
ax2.plot(time_grid, exp_decay(time_grid, *popt), '--', 
         linewidth=2, color='black', label=f'Fit: T₂* = {popt[1]:.2f} ps')
ax2.legend(fontsize=12)

plt.tight_layout()
plt.show()

## 6. Export Results

Save simulation results in various formats.

In [ ]:
# Export to JSON
framework.export_results(result, "graphene_simulation.json", format="json")
print("Results exported to: graphene_simulation.json")

# Export to HDF5 (if available)
try:
    framework.export_results(result, "graphene_simulation.h5", format="hdf5")
    print("Results exported to: graphene_simulation.h5")
except:
    print("HDF5 export not available")

# Export coherence times to CSV
import pandas as pd

df = pd.DataFrame({
    'Material': ['graphene'],
    'T1_ps': [coherence.T1_ps],
    'T2star_ps': [coherence.T2_star_ps],
    'T2_ps': [coherence.T2_echo_ps]
})

df.to_csv('coherence_times.csv', index=False)
print("Coherence times exported to: coherence_times.csv")

## Next Steps

- Try different materials: MoS2, WS2, hBN, etc.
- Vary temperature and system parameters
- Compare CPU vs GPU performance
- Explore advanced features: custom spectral densities, adaptive truncation

See the [documentation](https://docs.pseudomode-framework.org) for more details.